In [1]:
import pandas as pd
from statistics import median
from typing import Union, List
from pandas import DataFrame
import matplotlib.pyplot as plt
import jax.numpy as jnp
from qdax.utils.plotting import plot_2d_map_elites_repertoire_for_pgfplots
from scipy.stats import ranksums

In [2]:
def line_plot(df: DataFrame, x: str, y: Union[List[str], str], groups: Union[List[str], str] = None,
              file_prefix: str = "", cols_joiner: str = "_", filename_joiner: str = "_"):
    if isinstance(y, str):
        y = [y]
    if isinstance(groups, str):
        groups = [groups]

    def q1(a):
        return a.quantile(0.25)

    def q3(b):
        return b.quantile(0.75)

    vals = dict([(key, [q1, q3, median]) for key in y])

    summary = df.groupby(groups + [x]).agg(vals)
    summary.columns = [cols_joiner.join(col) for col in summary.columns.to_flat_index()]
    summary.reset_index(inplace=True)

    key_df = df.drop_duplicates(subset=groups)

    for i in range(len(key_df)):
        tmp = summary
        current_filename = file_prefix
        for key in groups:
            tmp = tmp[tmp[key] == key_df[key].iloc[i]]
            current_filename += f"{filename_joiner if len(current_filename) > 0 and not current_filename.endswith('/') else ''}{key_df[key].iloc[i]}"
        tmp.to_csv(f"{current_filename}.txt", sep="\t", index=False)


def box_plot(df: DataFrame, x: str, y: str, groups: Union[List[str], str] = None, file_prefix: str = "",
             filename_joiner: str = "_"):
    if isinstance(groups, str):
        groups = [groups]
    if groups is None or len(groups) == 0:
        _box_plot(df, x, y, file_prefix)

    else:
        key_df = df.drop_duplicates(subset=groups)

        for i in range(len(key_df)):
            tmp = df
            current_filename = file_prefix
            for key in groups:
                tmp = tmp[tmp[key] == key_df[key].iloc[i]]
                current_filename += f"{filename_joiner if len(current_filename) > 0 else ''}{key_df[key].iloc[i]}"
            _box_plot(tmp, x, y, current_filename)


def _box_plot(df: DataFrame, x: str, y: str, file_name: str):
    plt.figure(visible=False)
    data = []
    for xi in df[x].unique():
        data.append([k for k in df[df[x] == xi][y] if str(k) != "nan"])

    bp = plt.boxplot(data, showmeans=False)

    minimums = [round(item.get_ydata()[0], 1) for item in bp['caps']][::2]
    q1 = [round(min(item.get_ydata()), 1) for item in bp['boxes']]
    medians = [item.get_ydata()[0] for item in bp['medians']]
    q3 = [round(max(item.get_ydata()), 1) for item in bp['boxes']]
    maximums = [round(item.get_ydata()[0], 1) for item in bp['caps']][1::2]

    rows = [df[x].unique().tolist(), minimums, q1, medians, q3, maximums]

    with open(f"{file_name}.txt", "w") as bp_file:
        for row in rows:
            bp_file.write("\t".join(map(str, row)) + "\n")

# RQ1

In [3]:
dfs = []
algo_mapping = {
    "all": "3b", "s1": "brain", "s2": "body", "s3": "behavior", "ga": "ga"
}
for seed in range(10):
    for controller in ["nn", "graph"]:
        for sampler in algo_mapping.keys():
            tmp_df = pd.read_csv(
                f"../paper_results/ga/evobb_{controller}_{seed}.csv") if sampler == "ga" else pd.read_csv(
                f"../paper_results/me/evobb_{controller}_{algo_mapping[sampler]}_{seed}.csv")
            tmp_df["algorithm"] = algo_mapping[sampler]
            tmp_df["seed"] = seed
            tmp_df["controller"] = controller
            dfs.append(tmp_df)
df = pd.concat(dfs)
df.head()

,iteration,max_fitness,qd_score1,qd_score2,qd_score3,coverage1,coverage2,coverage3,time,current_time,invalid_individuals,algorithm,seed,controller
0,1,0.562116,5.652280,3.231952,2.908106,2.929688,6.640625,2.246094,11.388924,2024-12-02 13:35:18.571949,0.0,3b,0,nn
1,2,0.562116,7.412463,4.919590,3.104527,3.125000,8.593750,2.734375,5.778678,2024-12-02 13:35:24.350963,0.0,3b,0,nn
2,3,0.562116,8.975650,8.650234,3.866097,3.222656,10.058594,2.929688,5.733377,2024-12-02 13:35:30.084858,0.0,3b,0,nn
3,4,0.609543,10.033884,11.207098,4.322552,3.320312,11.523438,2.929688,5.646115,2024-12-02 13:35:35.731445,0.0,3b,0,nn
4,5,0.609543,10.713839,14.359545,4.868412,3.320312,12.304688,3.222656,5.769140,2024-12-02 13:35:41.501045,0.0,3b,0,nn


In [4]:
final_df = df[df["iteration"] == max(df["iteration"])]
final_df.head()

,iteration,max_fitness,qd_score1,qd_score2,qd_score3,coverage1,coverage2,coverage3,time,current_time,invalid_individuals,algorithm,seed,controller
3999,4000,10.449953,4718.811500,2498.80710,2663.113300,100.000000,52.441406,56.347656,6.022056,2024-12-02 20:07:36.555792,0.0,3b,0,nn
3999,4000,10.353656,5895.022500,1494.78560,1559.246500,100.000000,38.964844,31.445312,5.983479,2024-12-05 18:15:59.600939,0.0,brain,0,nn
3999,4000,9.384449,175.464770,3203.30130,1308.761400,5.468750,56.250000,27.929688,6.026099,2024-12-06 00:55:42.796596,0.0,body,0,nn
3999,4000,0.879594,3.826142,161.49133,-14.039587,2.832031,40.527344,12.011719,6.015054,2024-12-06 07:36:13.597999,0.0,behavior,0,nn
3999,4000,10.696037,NaN,NaN,NaN,NaN,NaN,NaN,4.224857,2024-08-09 21:38:00.419680,NaN,ga,0,nn


In [5]:
pairs = []
for sam in final_df.algorithm.unique():
    tmp_df = final_df[final_df["algorithm"] == sam]
    fits1 = tmp_df[tmp_df["controller"] == "nn"]["max_fitness"].to_list()
    fits2 = tmp_df[tmp_df["controller"] != "nn"]["max_fitness"].to_list()
    _, p_value = ranksums(fits1, fits2)
    print(f"{sam} -> {p_value}")

fits1 = final_df[(final_df["controller"] == "nn") & (final_df["algorithm"] == "ga")]["max_fitness"].to_list()
fits2 = final_df[(final_df["controller"] != "nn") & (final_df["algorithm"] == "3b")]["max_fitness"].to_list()
_, p_value = ranksums(fits1, fits2)
print(f"NN+GA vs graph+3B-QD -> {p_value}")

3b -> 0.3257513544787166
brain -> 0.1124105846553637
body -> 0.9397429895770734
behavior -> 0.0019397281129030408
ga -> 0.04125001659393949
NN+GA vs graph+3B-QD -> 0.3643461266335529


In [6]:
# evolution of fitness lineplots
line_plot(
    df=df,
    x="iteration",
    y="max_fitness",
    groups=["controller", "algorithm"],
    file_prefix="../pgfplots/evolution_fitness"
)

In [7]:
# end of evolution performance
box_plot(
    df=final_df,
    x="algorithm",
    y="max_fitness",
    groups="controller",
    file_prefix="../pgfplots/final_fitness"
)

<IPython.core.display.Javascript object>

/home/giorgianadizar/miniconda3/envs/qdaxpy38/lib/python3.8/site-packages/matplotlib/backends/backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


<IPython.core.display.Javascript object>

/home/giorgianadizar/miniconda3/envs/qdaxpy38/lib/python3.8/site-packages/matplotlib/backends/backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


In [8]:
# evolution of coverage lineplots 
line_plot(
    df=df[df["algorithm"] != "ga"],
    x="iteration",
    y=["coverage1", "coverage2", "coverage3"],
    groups=["controller", "algorithm"],
    file_prefix="../pgfplots/evolution_coverage"
)

In [9]:
# nn repertoire
rep_seed = 1
reps = {0: "brain", 1: "body", 2: "behavior"}
min_fit, max_fit = jnp.inf, -jnp.inf
base_path = f"../paper_results/me/evobb_nn_3b_{rep_seed}"
for r in reps.keys():
    fitnesses = jnp.load(f"{base_path}/r{r + 1}_fitnesses.npy")
    curr_min_fit, curr_max_fit = min(fitnesses[fitnesses > -jnp.inf]), max(fitnesses)
    if curr_min_fit < min_fit: min_fit = curr_min_fit
    if curr_max_fit > max_fit: max_fit = curr_max_fit

for r in reps.keys():
    centroids = jnp.load(f"{base_path}/r{r + 1}_centroids.npy")
    fitnesses = jnp.load(f"{base_path}/r{r + 1}_fitnesses.npy")
    v_min, v_max = plot_2d_map_elites_repertoire_for_pgfplots(
        centroids=centroids,
        repertoire_fitnesses=fitnesses,
        minval=jnp.asarray([0, 0]),
        maxval=jnp.asarray([1, 1]),
        vmin=min_fit,
        vmax=max_fit,
        target_file=f"../pgfplots/repertoire_nn_{reps[r]}.pdf"
    )
print(min_fit, max_fit)

<IPython.core.display.Javascript object>

/home/giorgianadizar/miniconda3/envs/qdaxpy38/lib/python3.8/site-packages/matplotlib/backends/backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-3.5918016 10.394796


In [10]:
# graph repertoire
rep_seed = 0
min_fit, max_fit = jnp.inf, -jnp.inf
base_path = f"../paper_results/me/evobb_graph_3b_{rep_seed}"
for r in reps.keys():
    fitnesses = jnp.load(f"{base_path}/r{r + 1}_fitnesses.npy")
    curr_min_fit, curr_max_fit = min(fitnesses[fitnesses > -jnp.inf]), max(fitnesses)
    if curr_min_fit < min_fit: min_fit = curr_min_fit
    if curr_max_fit > max_fit: max_fit = curr_max_fit

for r in reps.keys():
    centroids = jnp.load(f"{base_path}/r{r + 1}_centroids.npy")
    fitnesses = jnp.load(f"{base_path}/r{r + 1}_fitnesses.npy")
    v_min, v_max = plot_2d_map_elites_repertoire_for_pgfplots(
        centroids=centroids,
        repertoire_fitnesses=fitnesses,
        minval=jnp.asarray([0, 0]),
        maxval=jnp.asarray([1, 1]),
        vmin=min_fit,
        vmax=max_fit,
        target_file=f"../pgfplots/repertoire_graph_{reps[r]}.pdf"
    )
print(min_fit, max_fit)

<IPython.core.display.Javascript object>

/home/giorgianadizar/miniconda3/envs/qdaxpy38/lib/python3.8/site-packages/matplotlib/backends/backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-3.0884957 10.415704


In [11]:
from bbbqd.body.body_descriptors import get_body_descriptor_extractor
from pathlib import Path
import yaml
from bbbqd.body.body_utils import compute_body_mask, compute_body_float_genome_length, compute_body_encoding_function
import numpy as np

seed = 0
descs = []
for sampling in ["ga", "3b", "brain", "body", "behavior"]:
    print(sampling, end=" ")
    if sampling != "ga":
        folder = f"../paper_results/me/evobb_nn_{sampling}_{seed}"
        fitnesses = jnp.load(f"{folder}/r1_fitnesses.npy")
        genotypes = jnp.load(f"{folder}/r1_genotypes.npy")
    else:
        folder = f"../paper_results/ga/evobb_nn_{seed}"
        fitnesses = jnp.load(f"{folder}/scores.npy")
        genotypes = jnp.load(f"{folder}/genotypes.npy")
    config = yaml.load(Path(f"{folder}/config.yaml").read_text(), Loader=yaml.FullLoader)
    config["body_descriptors"] = ["relative_activity", "elongation"]
    body_encoding_fn = compute_body_encoding_function(config)
    best_genotype = genotypes[jnp.argmax(fitnesses)]
    body_mask_length = len(compute_body_mask(config))
    body_float_length = compute_body_float_genome_length(config)
    body_genome, controller_genome = jnp.split(best_genotype, [body_mask_length + body_float_length])
    body = body_encoding_fn(body_genome)
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=0)), axis=1)  # remove empty columns
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=1)), axis=0)  # remove empty rows
    h, w = body.shape
    body_string = "-".join("".join(map(str, r)) for r in body)
    body_descr_fn, _ = get_body_descriptor_extractor(config)
    body_descriptors = body_descr_fn(body)
    descs.append(body_descriptors)
    latex_command = f"\\vsrevogym{{{w}}}{{{h}}}{{{body_string}}}"
    print(latex_command)
for d in descs:
    print(f"$({d[0]:.2f},{d[1]:.2f})$", end= " & ")

ga \vsrevogym{4}{9}{4000-1300-4111-4140-0240-2040-4240-3400-2000}
3b \vsrevogym{8}{7}{24400400-30420311-00220400-00343400-00100000-00300000-00400000}
brain \vsrevogym{8}{8}{00000030-00000043-00000041-00000040-00004040-00111440-14210100-00210000}
body \vsrevogym{7}{7}{0200000-3230140-0042314-0432031-0400000-1400000-1000000}
behavior \vsrevogym{5}{8}{01220-01020-00031-00434-02100-01140-01040-11300}
$(0.55,0.56)$ & $(0.65,0.12)$ & $(0.50,0.45)$ & $(0.55,0.22)$ & $(0.35,0.44)$ & 

In [12]:
seed = 0
descs = []
for sampling in ["ga", "3b", "brain", "body", "behavior"]:
    print(sampling, end=" ")
    if sampling != "ga":
        folder = f"../paper_results/me/evobb_graph_{sampling}_{seed}"
        fitnesses = jnp.load(f"{folder}/r1_fitnesses.npy")
        genotypes = jnp.load(f"{folder}/r1_genotypes.npy")
    else:
        folder = f"../paper_results/ga/evobb_graph_{seed}"
        fitnesses = jnp.load(f"{folder}/scores.npy")
        genotypes = jnp.load(f"{folder}/genotypes.npy")
    config = yaml.load(Path(f"{folder}/config.yaml").read_text(), Loader=yaml.FullLoader)
    config["body_descriptors"] = ["relative_activity", "elongation"]
    body_encoding_fn = compute_body_encoding_function(config)
    best_genotype = genotypes[jnp.argmax(fitnesses)]
    body_mask_length = len(compute_body_mask(config))
    body_float_length = compute_body_float_genome_length(config)
    body_genome, controller_genome = jnp.split(best_genotype, [body_mask_length + body_float_length])
    body = body_encoding_fn(body_genome)
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=0)), axis=1)  # remove empty columns
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=1)), axis=0)  # remove empty rows
    h, w = body.shape
    body_string = "-".join("".join(map(str, r)) for r in body)
    body_descr_fn, _ = get_body_descriptor_extractor(config)
    body_descriptors = body_descr_fn(body)
    descs.append(body_descriptors)
    latex_command = f"\\vsrevogym{{{w}}}{{{h}}}{{{body_string}}}"
    print(latex_command)
for d in descs:
    print(f"$({d[0]:.2f},{d[1]:.2f})$", end= " & ")

ga \vsrevogym{7}{8}{0000330-0000411-0000140-0001140-1411340-0040040-0040000-0040000}
3b \vsrevogym{8}{5}{21210000-03410000-02330430-00013331-00000443}
brain \vsrevogym{6}{6}{002003-004303-010341-030240-221240-102200}
body \vsrevogym{6}{7}{012000-002130-003411-212400-013400-041200-010000}
behavior \vsrevogym{8}{7}{00000020-00000040-00210023-04200040-03324440-04200400-24000000}
$(0.60,0.33)$ & $(0.60,0.56)$ & $(0.45,0.38)$ & $(0.35,0.22)$ & $(0.60,0.45)$ & 

# RQ2

In [13]:
tasks_extra_data = {
    "BridgeWalker-v0": ("walking", 6.6),
    "BidirectionalWalker-v0": ("walking", 8.7),

    "CustomCarrier-v0": ("object", 10.8),
    "CustomPusher-v0": ("object", 9.5),
    "Pusher-v1": ("object", 8.2),
    "Catcher-v0": ("object", -1.2),
    "BeamToppler-v0": ("object", 9.2),

    "UpStepper-v0": ("locomotion", 7.8),
    "DownStepper-v0": ("locomotion", 9.1),
    "ObstacleTraverser-v0": ("locomotion", 9.5),
    "ObstacleTraverser-v1": ("locomotion", 5.1),
    "Hurdler-v0": ("locomotion", 4.2),
    "PlatformJumper-v0": ("locomotion", 5.8),
    "GapJumper-v0": ("locomotion", 7.8),
    "Traverser-v0": ("locomotion", 5.9),
    "CaveCrawler-v0": ("locomotion", 5.6),

    "AreaMaximizer-v0": ("shape", 2.8),
    "AreaMinimizer-v0": ("shape", 1),
    "WingspanMazimizer-v0": ("shape", .87),
    "HeightMaximizer-v0": ("shape", .5),

    "Flipper-v0": ("misc", 48.3),
    "Jumper-v0": ("misc", 4.9),
    "Balancer-v0": ("misc", .16),
}
# other_tasks_extra_data = {
#     "Walker-v0": ("walking", 10.6),
#     "Thrower-v0": ("object", None),
#     "Carrier-v1": ("object", None),
#     "Slider-v0": ("object", None),
#     "Lifter-v0": ("object", None),
#
#     # all climbers
#     "Climber-v0": ("climb", None),
#     "Climber-v1": ("climb", None),
#     "Climber-v2": ("climb", None),
#
#     "Balancer-v1": ("misc", None)
# }

In [14]:
tasks = dict(tasks_extra_data)
# tasks.update(other_tasks_extra_data)
tasks_df_dicts = []
for task_name, task_extra_info in tasks.items():
    tasks_df_dicts.append({
        "task": task_name,
        "type": task_extra_info[0],
        "max_reward": task_extra_info[1],
    })
tasks_df = pd.DataFrame(tasks_df_dicts)
tasks_df.head()

,task,type,max_reward
0,BridgeWalker-v0,walking,6.6
1,BidirectionalWalker-v0,walking,8.7
2,CustomCarrier-v0,object,10.8
3,CustomPusher-v0,object,9.5
4,Pusher-v1,object,8.2


In [15]:
transfer_dfs = []
samplings = algo_mapping.values()
repertoires = [f"g{i + 1}" for i in range(3)]
for task in tasks:
    for controller in ["graph", "nn"]:
        for seed in range(10):
            for sampler in samplings:
                if sampler == "ga":
                    tmp_df = pd.read_csv(f"../paper_results/ga_transfer/evobb_{controller}_{seed}_{task}.csv")
                    tmp_df["seed"] = seed
                    tmp_df["task"] = task
                    tmp_df["algorithm"] = "ga"
                    tmp_df["repertoire"] = "all"
                    tmp_df["controller"] = controller
                    tmp_df["max_fitness"] = tmp_df["max_fitness"].apply(
                        lambda x: float(x.replace("[", "").replace("]", "")))
                    transfer_dfs.append(tmp_df)
                else:
                    for rep in repertoires:
                        tmp_df = pd.read_csv(
                            f"../paper_results/me_transfer/evobb_{controller}_{sampler}_{seed}_{rep}_{task}.csv")
                        tmp_df["seed"] = seed
                        tmp_df["task"] = task
                        tmp_df["algorithm"] = sampler
                        tmp_df["repertoire"] = rep
                        tmp_df["controller"] = controller
                        # tmp_df["max_fitness"] = tmp_df["max_fitness"].apply(lambda x: float(x.replace("[", "").replace("]", "")))
                        transfer_dfs.append(tmp_df)

transfer_df = pd.concat(transfer_dfs, ignore_index=True)

In [16]:
max_transfer = transfer_df.groupby(["task", "controller"]).max("max_fitness").reset_index()[
    ["task", "controller", "max_fitness"]]
max_transfer_pivot = max_transfer.pivot(index='task', columns='controller', values='max_fitness')
max_transfer_pivot = max_transfer_pivot.rename(columns={'graph': 'max_fitness_cgp', 'nn': 'max_fitness_nn'}).reset_index()
max_transfer_pivot.head()

controller,task,max_fitness_cgp,max_fitness_nn
0,AreaMaximizer-v0,1.520370,1.531976
1,AreaMinimizer-v0,1.354250,1.597692
2,Balancer-v0,0.709250,0.672029
3,BeamToppler-v0,9.379107,9.476809
4,BidirectionalWalker-v0,1.512540,1.032905


In [17]:
tasks_and_transfer_df = pd.merge(tasks_df, max_transfer_pivot, on="task", how="left")
tasks_and_transfer_df["max_cgp_percentage"] = tasks_and_transfer_df["max_fitness_cgp"] / tasks_and_transfer_df[
    "max_reward"]
tasks_and_transfer_df["max_nn_percentage"] = tasks_and_transfer_df["max_fitness_nn"] / tasks_and_transfer_df[
    "max_reward"]
tasks_and_transfer_df = tasks_and_transfer_df[
    ["type", "task", "max_reward", "max_fitness_cgp", "max_fitness_nn"]]
tasks_and_transfer_df.sort_values(by=["type"], inplace=True)
tasks_and_transfer_df.head()

,type,task,max_reward,max_fitness_cgp,max_fitness_nn
11,locomotion,Hurdler-v0,4.2,3.628619,3.475937
15,locomotion,CaveCrawler-v0,5.6,5.498639,5.869548
14,locomotion,Traverser-v0,5.9,7.550612,4.662922
13,locomotion,GapJumper-v0,7.8,6.378554,5.034650
12,locomotion,PlatformJumper-v0,5.8,7.538194,5.903883


In [18]:
print(tasks_and_transfer_df.to_latex(index=False, float_format="{:0.2f}".format))

\begin{tabular}{llrrr}
\toprule
type & task & max_reward & max_fitness_cgp & max_fitness_nn \\
\midrule
locomotion & Hurdler-v0 & 4.20 & 3.63 & 3.48 \\
locomotion & CaveCrawler-v0 & 5.60 & 5.50 & 5.87 \\
locomotion & Traverser-v0 & 5.90 & 7.55 & 4.66 \\
locomotion & GapJumper-v0 & 7.80 & 6.38 & 5.03 \\
locomotion & PlatformJumper-v0 & 5.80 & 7.54 & 5.90 \\
locomotion & UpStepper-v0 & 7.80 & 6.20 & 4.48 \\
locomotion & DownStepper-v0 & 9.10 & 9.07 & 9.09 \\
locomotion & ObstacleTraverser-v0 & 9.50 & 9.46 & 9.49 \\
locomotion & ObstacleTraverser-v1 & 5.10 & 4.92 & 5.04 \\
misc & Flipper-v0 & 48.30 & 3.54 & 1.87 \\
misc & Jumper-v0 & 4.90 & 0.11 & 0.02 \\
misc & Balancer-v0 & 0.16 & 0.71 & 0.67 \\
object & Catcher-v0 & -1.20 & 1.04 & 1.12 \\
object & Pusher-v1 & 8.20 & 0.20 & 0.24 \\
object & CustomPusher-v0 & 9.50 & 13.15 & 11.99 \\
object & CustomCarrier-v0 & 10.80 & 10.57 & 10.86 \\
object & BeamToppler-v0 & 9.20 & 9.38 & 9.48 \\
shape & AreaMaximizer-v0 & 2.80 & 1.52 & 1.53 \\
shape &

In [19]:
solved_tasks = [
    "BridgeWalker-v0",
    "UpStepper-v0",
    "DownStepper-v0",
    "ObstacleTraverser-v0",
    "ObstacleTraverser-v1",
    "Hurdler-v0",
    "PlatformJumper-v0",
    "GapJumper-v0",
    "Traverser-v0",
    "CaveCrawler-v0",
    "CustomCarrier-v0",
    "CustomPusher-v0",
    "Catcher-v0",
    "BeamToppler-v0",
    "AreaMinimizer-v0",
    "WingspanMazimizer-v0",
    "HeightMaximizer-v0",
    "Balancer-v0",
]
tasks_aliases = ["W1", "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8", "L9", "O1", "O2", "O3", "O4", "S1", "S2", "S3",
                 "M1"]
tasks_dict = {}
for i in range(len(solved_tasks)):
    tasks_dict[solved_tasks[i]] = tasks_aliases[i]
tasks_dict

{'BridgeWalker-v0': 'W1',
 'UpStepper-v0': 'L1',
 'DownStepper-v0': 'L2',
 'ObstacleTraverser-v0': 'L3',
 'ObstacleTraverser-v1': 'L4',
 'Hurdler-v0': 'L5',
 'PlatformJumper-v0': 'L6',
 'GapJumper-v0': 'L7',
 'Traverser-v0': 'L8',
 'CaveCrawler-v0': 'L9',
 'CustomCarrier-v0': 'O1',
 'CustomPusher-v0': 'O2',
 'Catcher-v0': 'O3',
 'BeamToppler-v0': 'O4',
 'AreaMinimizer-v0': 'S1',
 'WingspanMazimizer-v0': 'S2',
 'HeightMaximizer-v0': 'S3',
 'Balancer-v0': 'M1'}

In [20]:
df_transferred_and_solved = transfer_df[transfer_df["task"].isin(solved_tasks)]
df_transferred_and_solved.head()

,max_fitness,coverage1,coverage2,coverage3,seed,task,algorithm,repertoire,controller
0,4.959429,4.882812,3.417969,2.050781,0,BridgeWalker-v0,3b,g1,graph
1,6.391897,0.292969,4.882812,1.269531,0,BridgeWalker-v0,3b,g2,graph
2,6.391897,0.390625,3.515625,1.660156,0,BridgeWalker-v0,3b,g3,graph
3,3.316065,4.882812,0.976562,0.878906,0,BridgeWalker-v0,brain,g1,graph
4,3.302593,3.027344,4.882812,1.464844,0,BridgeWalker-v0,brain,g2,graph


In [23]:
df_transferred_and_solved_nn = df_transferred_and_solved[df_transferred_and_solved["controller"] == "nn"]
df_best_transfer_grouped_nn = df_transferred_and_solved_nn.loc[
    df_transferred_and_solved_nn.groupby(["task", "algorithm", "seed"])["max_fitness"].idxmax()]
df_best_transfer_grouped_nn["rank"] = df_best_transfer_grouped_nn.groupby(["task"])["max_fitness"].rank(
    method="dense",
    ascending=True) / 50.
df_best_transfer_nn = \
    df_best_transfer_grouped_nn.groupby(["task", "algorithm"])["rank"].mean().reset_index()

for samp in ["ga", "3b", "brain", "body", "behavior"]:
    values = []
    for task in solved_tasks:
        v = df_best_transfer_nn[(df_best_transfer_nn["task"] == task) & (df_best_transfer_nn["algorithm"] == samp)][
            "rank"].tolist()[0]
        values.append(10 * v)
    print(samp, values)

ga [5.380000000000001, 2.46, 1.9, 2.8200000000000003, 6.18, 2.78, 4.18, 6.66, 7.1, 5.4, 4.62, 5.0, 1.2, 4.46, 2.88, 5.32, 1.4, 3.1]
3b [7.22, 6.08, 7.34, 6.22, 5.12, 6.5, 6.5, 5.0600000000000005, 5.6, 6.12, 6.68, 6.58, 4.16, 5.860000000000001, 4.6000000000000005, 3.14, 6.54, 3.5999999999999996]
brain [6.58, 6.6000000000000005, 7.119999999999999, 6.68, 6.18, 6.38, 5.540000000000001, 5.74, 5.68, 6.4, 5.88, 6.98, 5.44, 7.039999999999999, 5.780000000000001, 3.7, 4.86, 5.64]
body [4.94, 6.639999999999999, 6.0, 6.9799999999999995, 5.76, 6.78, 6.3, 6.08, 5.700000000000001, 6.42, 6.799999999999999, 5.72, 6.24, 6.639999999999999, 5.56, 7.08, 5.48, 4.9]
behavior [1.38, 3.7199999999999998, 3.14, 2.8, 2.26, 3.06, 2.98, 1.96, 1.42, 1.1600000000000001, 1.52, 1.22, 8.46, 1.5, 6.68, 6.26, 7.22, 8.26]


In [27]:
df_transferred_and_solved_cgp = df_transferred_and_solved[df_transferred_and_solved["controller"] == "graph"]
df_best_transfer_grouped_cgp = df_transferred_and_solved_cgp.loc[
    df_transferred_and_solved_cgp.groupby(["task", "algorithm", "seed"])["max_fitness"].idxmax()]
df_best_transfer_grouped_cgp["rank"] = df_best_transfer_grouped_cgp.groupby(["task"])["max_fitness"].rank(
    method="dense",
    ascending=True) / 50.
df_best_transfer_cgp = \
    df_best_transfer_grouped_cgp.groupby(["task", "algorithm"])["rank"].mean().reset_index()

for task in tasks_aliases:
    print(f"{task},", end="")
print()

for samp in ["ga", "3b", "brain", "body", "behavior"]:
    values = []
    for task in solved_tasks:
        v = df_best_transfer_cgp[(df_best_transfer_cgp["task"] == task) & (df_best_transfer_cgp["algorithm"] == samp)][
            "rank"].tolist()[0]
        values.append(10 * v)
    print(samp, values)

W1,L1,L2,L3,L4,L5,L6,L7,L8,L9,O1,O2,O3,O4,S1,S2,S3,M1,
ga [1.98, 1.98, 1.3, 1.34, 2.38, 1.32, 1.1400000000000001, 2.02, 2.72, 1.42, 1.7599999999999998, 1.6599999999999997, 1.7799999999999998, 1.38, 1.16, 1.12, 1.8599999999999999, 1.28]
3b [6.34, 7.98, 7.640000000000001, 8.08, 7.82, 7.16, 7.74, 7.359999999999999, 7.4399999999999995, 5.84, 7.18, 6.98, 5.08, 6.68, 6.959999999999999, 5.96, 6.82, 5.48]
brain [5.300000000000001, 4.14, 5.18, 4.62, 3.58, 3.9000000000000004, 4.28, 4.62, 4.18, 5.42, 5.140000000000001, 6.08, 5.16, 5.26, 5.58, 6.38, 3.9800000000000004, 4.5]
body [6.799999999999999, 6.4, 5.98, 6.02, 6.38, 6.94, 6.2, 6.0, 5.34, 7.039999999999999, 5.74, 6.08, 6.92, 6.66, 6.5600000000000005, 7.24, 5.540000000000001, 7.76]
behavior [5.08, 5.0, 5.4, 5.44, 5.34, 6.18, 6.140000000000001, 5.5, 5.82, 5.779999999999999, 5.68, 4.7, 6.559999999999999, 5.52, 5.24, 4.8, 7.3, 6.48]


In [28]:
d_nn = pd.read_csv("../paper_results/me_transfer/original_descriptors_nn.csv")
d_nn["controller"] = "nn"
d_cgp = pd.read_csv("../paper_results/me_transfer/original_descriptors_graph.csv")
d_cgp["controller"] = "cgp"
original_d_tmp = pd.concat([d_nn, d_cgp], ignore_index=True)
original_d_tmp = original_d_tmp[original_d_tmp["environment"].isin(solved_tasks)]
original_d_tmp = original_d_tmp[original_d_tmp["sampler"] == "all"]
best_provenience = original_d_tmp.copy()
best_provenience['all'] = (
        (best_provenience['brain_descriptors_0'] >= 0) & (best_provenience['body_descriptors_0'] >= 0) & (
        best_provenience['behavior_descriptors_0'] >= 0)).astype(int)
best_provenience['brain+body'] = (
        (best_provenience['brain_descriptors_0'] >= 0) & (best_provenience['body_descriptors_0'] >= 0) & (
        best_provenience['behavior_descriptors_0'] < 0)).astype(int)
best_provenience['brain+behavior'] = (
        (best_provenience['brain_descriptors_0'] >= 0) & (best_provenience['body_descriptors_0'] < 0) & (
        best_provenience['behavior_descriptors_0'] >= 0)).astype(int)
best_provenience['body+behavior'] = (
        (best_provenience['brain_descriptors_0'] < 0) & (best_provenience['body_descriptors_0'] >= 0) & (
        best_provenience['behavior_descriptors_0'] >= 0)).astype(int)
best_provenience['brain'] = (
        (best_provenience['brain_descriptors_0'] >= 0) & (best_provenience['body_descriptors_0'] < 0) & (
        best_provenience['behavior_descriptors_0'] < 0)).astype(int)
best_provenience['body'] = (
        (best_provenience['brain_descriptors_0'] < 0) & (best_provenience['body_descriptors_0'] >= 0) & (
        best_provenience['behavior_descriptors_0'] < 0)).astype(int)
best_provenience['behavior'] = (
        (best_provenience['brain_descriptors_0'] < 0) & (best_provenience['body_descriptors_0'] < 0) & (
        best_provenience['behavior_descriptors_0'] >= 0)).astype(int)

In [29]:
best_provenience_grouped = best_provenience.groupby('controller')[
    ['all', 'brain+body', 'body+behavior', 'brain+behavior', 'brain', 'body', 'behavior']].sum()
best_provenience_grouped

,all,brain+body,body+behavior,brain+behavior,brain,body,behavior
controller,,,,,,,
cgp,7,1,48,4,57,28,35
nn,31,20,22,14,33,27,33


In [30]:
id_cols = ["sampler", "seed", "environment", "controller"]
melt_cols = [col for col in original_d_tmp.columns if col not in id_cols]
original_d_melted = original_d_tmp.melt(id_vars=id_cols, value_vars=melt_cols, var_name="category", value_name="value")
original_d_melted["repertoire"] = original_d_melted["category"].str.extract(r'([^_]+)')
original_d_melted["descriptor_id"] = original_d_melted["category"].str.extract(r'(\d)').astype(int)
original_descriptors = original_d_melted.pivot(index=id_cols + ["repertoire"], columns="descriptor_id",
                                               values="value").reset_index()
original_descriptors.columns = id_cols + ["repertoire", "x", "y"]
original_descriptors = original_descriptors[["x", "y", "repertoire"] + id_cols]
original_descriptors['task'] = original_descriptors['environment'].map(tasks_dict)
original_descriptors

,x,y,repertoire,sampler,seed,environment,controller,task
0,-1.000000,-1.000000,behavior,all,0,AreaMinimizer-v0,cgp,S1
1,-1.000000,-1.000000,body,all,0,AreaMinimizer-v0,cgp,S1
2,0.080000,0.220000,brain,all,0,AreaMinimizer-v0,cgp,S1
3,0.144578,0.717742,behavior,all,0,AreaMinimizer-v0,nn,S1
4,0.650000,0.125000,body,all,0,AreaMinimizer-v0,nn,S1
...,...,...,...,...,...,...,...,...
1075,-1.000000,-1.000000,body,all,9,WingspanMazimizer-v0,cgp,S2
1076,0.040000,0.140000,brain,all,9,WingspanMazimizer-v0,cgp,S2
1077,0.358566,0.634000,behavior,all,9,WingspanMazimizer-v0,nn,S2
1078,0.300000,0.500000,body,all,9,WingspanMazimizer-v0,nn,S2


In [31]:
filtered_descriptors = original_descriptors[(original_descriptors["x"] >= 0) & (original_descriptors["y"] >= 0)]
for controller in original_descriptors["controller"].unique():
    for repertoire in original_descriptors["repertoire"].unique():
        tmp_df = filtered_descriptors[
            (filtered_descriptors["controller"] == controller) & (filtered_descriptors["repertoire"] == repertoire)][
            ["x", "y", "task"]]
        tmp_df.to_csv(f"../pgfplots/scatter_{controller}_{repertoire}.txt", sep="\t", index=False)

In [32]:
for t in original_descriptors["task"].unique():
    print(f"{t}={{{t.lower()}}},", end=" ")

S1={s1}, M1={m1}, O4={o4}, W1={w1}, O3={o3}, L9={l9}, O1={o1}, O2={o2}, L2={l2}, L7={l7}, S3={s3}, L5={l5}, L3={l3}, L4={l4}, L6={l6}, L8={l8}, L1={l1}, S2={s2}, 

In [33]:
for t in tasks_aliases:
    print("\\addlegendimageintext{mark legend image={" + t.lower() + "}{*}} " + t + " \hspace{5mm}")

\addlegendimageintext{mark legend image={w1}{*}} W1 \hspace{5mm}
\addlegendimageintext{mark legend image={l1}{*}} L1 \hspace{5mm}
\addlegendimageintext{mark legend image={l2}{*}} L2 \hspace{5mm}
\addlegendimageintext{mark legend image={l3}{*}} L3 \hspace{5mm}
\addlegendimageintext{mark legend image={l4}{*}} L4 \hspace{5mm}
\addlegendimageintext{mark legend image={l5}{*}} L5 \hspace{5mm}
\addlegendimageintext{mark legend image={l6}{*}} L6 \hspace{5mm}
\addlegendimageintext{mark legend image={l7}{*}} L7 \hspace{5mm}
\addlegendimageintext{mark legend image={l8}{*}} L8 \hspace{5mm}
\addlegendimageintext{mark legend image={l9}{*}} L9 \hspace{5mm}
\addlegendimageintext{mark legend image={o1}{*}} O1 \hspace{5mm}
\addlegendimageintext{mark legend image={o2}{*}} O2 \hspace{5mm}
\addlegendimageintext{mark legend image={o3}{*}} O3 \hspace{5mm}
\addlegendimageintext{mark legend image={o4}{*}} O4 \hspace{5mm}
\addlegendimageintext{mark legend image={s1}{*}} S1 \hspace{5mm}
\addlegendimageintext{mar

<>:2: DeprecationWarning: invalid escape sequence \h


In [34]:
# subtasks = ["W1", "L1", "O1", "S1", "M1"]
seed = 0
sampling = "3b"
print(" & ".join(tasks_aliases))
for task in solved_tasks:
    # print(task, end=" ")
    folder = f"../paper_results/me_transfer/evobb_nn_{sampling}_{seed}_g1_{task}"
    fitnesses = jnp.load(f"{folder}/r1_fitnesses.npy")
    genotypes = jnp.load(f"{folder}/r1_genotypes.npy")
    config = yaml.load(Path(f"{folder}/config.yaml").read_text(), Loader=yaml.FullLoader)
    config["body_descriptors"] = ["relative_activity", "elongation"]
    body_encoding_fn = compute_body_encoding_function(config)
    best_genotype = genotypes[jnp.argmax(fitnesses)]
    body_mask_length = len(compute_body_mask(config))
    body_float_length = compute_body_float_genome_length(config)
    body_genome, controller_genome = jnp.split(best_genotype, [body_mask_length + body_float_length])
    body = body_encoding_fn(body_genome)
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=0)), axis=1)  # remove empty columns
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=1)), axis=0)  # remove empty rows
    h, w = body.shape
    body_string = "-".join("".join(map(str, r)) for r in body)
    body_descr_fn, _ = get_body_descriptor_extractor(config)
    body_descriptors = body_descr_fn(body)
    descs.append(body_descriptors)
    latex_command = f"\\vsrevogym{{{w}}}{{{h}}}{{{body_string}}}"
    print(latex_command, end=" & ")
# for d in descs:
#     print(f"$({d[0]:.2f},{d[1]:.2f})$", end= " & ")

W1 & L1 & L2 & L3 & L4 & L5 & L6 & L7 & L8 & L9 & O1 & O2 & O3 & O4 & S1 & S2 & S3 & M1
\vsrevogym{7}{7}{0000034-0040430-0220400-0422400-0011400-3212000-0200000} & \vsrevogym{7}{7}{0000034-0040430-0220400-0422400-0011400-3212000-0200000} & \vsrevogym{7}{7}{0000044-0040430-0220400-0122400-1002400-3442000-0400000} & \vsrevogym{8}{7}{00000022-00040240-00040400-00142400-01001400-21421000-00200000} & \vsrevogym{7}{6}{0003432-0031440-0001300-1110400-1041400-4002000} & \vsrevogym{7}{6}{0003432-0031440-0001300-1110400-1041400-4002000} & \vsrevogym{6}{7}{002412-032340-001300-310400-044400-422000-100000} & \vsrevogym{7}{6}{0003432-0031430-0001200-3110400-1041400-4002000} & \vsrevogym{7}{6}{0003432-0031430-0001200-3110400-1041400-4002000} & \vsrevogym{8}{7}{00000022-00000320-00003400-00122400-01021400-24121000-00200000} & \vsrevogym{8}{7}{00000022-00000140-00020400-10022400-42001400-24121000-00200000} & \vsrevogym{7}{6}{0003432-0031430-0001200-3110400-1041400-4002000} & \vsrevogym{6}{7}{002412-03

In [36]:
# subtasks = ["W1", "L1", "O1", "S1", "M1"]
seed = 0
sampling = "3b"
for task in solved_tasks:
    # print(task, end=" ")
    folder = f"../paper_results/me_transfer/evobb_graph_{sampling}_{seed}_g1_{task}"
    fitnesses = jnp.load(f"{folder}/r1_fitnesses.npy")
    genotypes = jnp.load(f"{folder}/r1_genotypes.npy")
    config = yaml.load(Path(f"{folder}/config.yaml").read_text(), Loader=yaml.FullLoader)
    config["body_descriptors"] = ["relative_activity", "elongation"]
    body_encoding_fn = compute_body_encoding_function(config)
    best_genotype = genotypes[jnp.argmax(fitnesses)]
    body_mask_length = len(compute_body_mask(config))
    body_float_length = compute_body_float_genome_length(config)
    body_genome, controller_genome = jnp.split(best_genotype, [body_mask_length + body_float_length])
    body = body_encoding_fn(body_genome)
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=0)), axis=1)  # remove empty columns
    body = np.delete(body, np.argwhere(np.all(body[..., :] == 0, axis=1)), axis=0)  # remove empty rows
    h, w = body.shape
    body_string = "-".join("".join(map(str, r)) for r in body)
    body_descr_fn, _ = get_body_descriptor_extractor(config)
    body_descriptors = body_descr_fn(body)
    descs.append(body_descriptors)
    latex_command = f"\\vsrevogym{{{w}}}{{{h}}}{{{body_string}}}"
    print(latex_command, end=" & ")
# for d in descs:
#     print(f"$({d[0]:.2f},{d[1]:.2f})$", end= " & ")

\vsrevogym{8}{5}{21210000-03410000-02330430-00013331-00000443} & \vsrevogym{8}{5}{21210000-03410000-02330430-00013331-00000443} & \vsrevogym{8}{5}{21210000-03410000-02330430-00013331-00000443} & \vsrevogym{8}{5}{21210000-03410000-02330430-00013331-00000443} & \vsrevogym{8}{5}{21210000-03410000-02330430-00013331-00000443} & \vsrevogym{6}{7}{232010-010040-240240-043243-440400-300000-300000} & \vsrevogym{7}{7}{0003230-0001000-0002010-0002114-0304400-1332400-0104000} & \vsrevogym{6}{6}{000431-001240-023302-242434-100034-000010} & \vsrevogym{6}{6}{030001-342014-034131-304400-432400-004000} & \vsrevogym{8}{5}{31201000-00111131-30442000-11240000-40400000} & \vsrevogym{8}{5}{21210000-03410000-02330430-00013331-00000443} & \vsrevogym{6}{7}{000042-032010-020040-340240-043243-440400-200000} & \vsrevogym{7}{7}{0003000-0001000-0002010-0002114-0324420-1332400-0104000} & \vsrevogym{7}{7}{0020243-0040440-0434340-0304000-3404000-0100000-4400000} & \vsrevogym{7}{7}{0003200-0001000-0002010-0002114-033042